# 🚀 Automated 2D to 360° Video Processing

This notebook automates the process of converting a standard 2D video into a 360° panoramic video using the 'Beyond the Frame' model. It handles all setup, downloads a video from a URL, and runs the command-line inference.

**Instructions:**
1. Make sure your runtime is set to use a GPU (`Runtime` > `Change runtime type` > `T4 GPU`).
2. **(Optional)** Change the `video_url` in the second code cell to your desired video (e.g., from YouTube).
3. Run each cell in order from top to bottom.

### Step 1: Clone the Repository and Install Dependencies

In [ ]:
!git clone --recurse-submodules https://github.com/Red-Fairy/argus-code
%cd argus-code

print('Installing dependencies...')
# Install faiss-gpu first, as it's sensitive to installation order.
!pip install -q faiss-gpu
!pip install -q -r requirements.txt
!pip install -q yt-dlp

print('Cloning and installing asmk...')
!git clone https://github.com/jenicek/asmk
%cd asmk/cython/
!cythonize *.pyx
%cd ..
!pip install .
%cd ..

### Step 2: Download Pre-trained Models

In [ ]:
print('Downloading pre-trained models...')
!mkdir -p checkpoints
!pip install -q gdown
!gdown --id 1mZpViQY2yvwav-CcxdsQouYu8t52b25G -O checkpoints/pretrained-weights.zip
!unzip -q checkpoints/pretrained-weights.zip -d checkpoints/

!mkdir -p mega-sam/Depth-Anything/checkpoints
!wget -q https://huggingface.co/spaces/LiheYoung/Depth-Anything/resolve/main/checkpoints/depth_anything_vitl14.pth -O mega-sam/Depth-Anything/checkpoints/depth_anything_vitl14.pth

!mkdir -p mega-sam/cvd_opt
!gdown --id 1sWDsfuZ3Up38EUQt7-JDTT1HcGHuJgvT -O mega-sam/cvd_opt/raft-things.pth
print('Models downloaded.')

### Step 3: Download Gameplay Video

In [ ]:
# --- IMPORTANT --- 
# You can change this URL to any video you want to process.
# For example, a YouTube link to GTA or Minecraft gameplay.
video_url = 'https://www.youtube.com/watch?v=YE7VzlLtp-4' # Big Buck Bunny

!mkdir -p input_videos
!yt-dlp -o 'input_videos/gameplay.mp4' -f 'best[height<=720]' {video_url}

video_path = 'input_videos/gameplay.mp4'
print(f'Video downloaded to: {video_path}')

### Step 4: Run Inference and Generate 360° Video
This step runs the main processing script. It will take the downloaded video as input and generate the 360° video in the `outputs/` directory.

In [ ]:
unet_path = './checkpoints/pretrained-weights'
save_folder = './outputs'
guidance_scale = 3
inference_steps = 25

!python inference.py \
    --unet_path {unet_path} \
    --video_path {video_path} \
    --val_save_folder {save_folder} \
    --guidance_scale {guidance_scale} \
    --num_inference_steps {inference_steps}

print(f'\n--- Processing Complete! ---\n')
print(f'Check the `{save_folder}` directory for your generated 360° video.')